## Connecting the Dots

In [ ]:
import tkinter as tk
import numpy as np
import math

class HoughTransformApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Connect Dots with Hough Transform")
        self.canvas = tk.Canvas(master, width=600, height=400, bg='white')
        self.canvas.pack()
        self.dots = []  
        self.canvas.bind("<Button-1>", self.place_dot)
        self.connect_button = tk.Button(master, text="Connect Dots", command=self.connect_dots)
        self.connect_button.pack()

    def place_dot(self, event):
        x, y = event.x, event.y
        self.dots.append((x, y))  
        self.canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill='red')  
        print(f"Dot placed at: ({x}, {y})")  

    def connect_dots(self):
        if len(self.dots) < 2:
            print("Need at least two dots to connect.")
            return
        
        lines = self.hough_transform(self.dots)
        for rho, theta in lines:
            self.draw_line(rho, theta)

    def hough_transform(self, dots):
        
        theta_range = np.linspace(-np.pi/2, np.pi/2, 180)  
        rho_max = int(np.hypot(600, 400))  
        rho_range = np.arange(-rho_max, rho_max, 1)  

        
        accumulator = np.zeros((len(rho_range), len(theta_range)), dtype=int)

        
        for (x, y) in dots:
            for theta_index, theta in enumerate(theta_range):
                rho = int(x * np.cos(theta) + y * np.sin(theta))
                rho_index = rho + rho_max  
                if 0 <= rho_index < len(rho_range):
                    accumulator[rho_index, theta_index] += 1

        
        threshold = 1  
        lines = []
        for i in range(len(rho_range)):
            for j in range(len(theta_range)):
                if accumulator[i, j] > threshold:
                    rho = rho_range[i]
                    theta = theta_range[j]
                    lines.append((rho, theta))
                    print(f"Detected line: (rho={rho}, theta={theta})")  

        return lines

    def draw_line(self, rho, theta):
       
        x0 = rho * np.cos(theta)
        y0 = rho * np.sin(theta)
      
        x1 = int(x0 + 1000 * (-np.sin(theta)))  
        y1 = int(y0 + 1000 * (np.cos(theta)))   
        x2 = int(x0 - 1000 * (-np.sin(theta)))  
        y2 = int(y0 - 1000 * (np.cos(theta)))    

       
        self.canvas.create_line(x1, y1, x2, y2, fill='blue')
        print(f"Line drawn from: ({x1}, {y1}) to ({x2}, {y2})")  

if __name__ == "__main__":
    root = tk.Tk()
    app = HoughTransformApp(root)
    root.mainloop()


Dot placed at: (306, 168)
Dot placed at: (201, 130)
Dot placed at: (171, 167)
Detected line: (rho=-167, theta=-1.5707963267948966)
Detected line: (rho=-52, theta=-1.2197803878463165)
Line drawn from: (1000, 167) to (-1000, 166)
Line drawn from: (921, 392) to (-956, -295)


### Detecting Edges

In [8]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np
import math

def upload_and_preprocess_image():
    global img, displayed_img, edge_img
    filepath = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if filepath:
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        edge_img = cv2.Canny(img, 100, 200)  
        display_image(edge_img)

def display_image(img_array):
    img_pil = Image.fromarray(img_array)
    img_tk = ImageTk.PhotoImage(img_pil)
    displayed_img.configure(image=img_tk)
    displayed_img.image = img_tk

def apply_custom_hough_transform():
    if edge_img is None:
        print("Please upload an image and detect edges first.")
        return
    
    width, height = edge_img.shape
    diag_len = int(math.sqrt(width**2 + height**2))
    theta_res = 1  
    rho_res = 1    

    theta_range = np.deg2rad(np.arange(-90, 90, theta_res))
    rho_range = np.arange(-diag_len, diag_len, rho_res)
    accumulator = np.zeros((len(rho_range), len(theta_range)), dtype=int)

    edge_points = np.argwhere(edge_img > 0)  
    for (y, x) in edge_points:
        for theta_idx, theta in enumerate(theta_range):
            rho = int(x * np.cos(theta) + y * np.sin(theta))
            rho_idx = int(rho + diag_len)  
            accumulator[rho_idx, theta_idx] += 1

    threshold = 100 
    lines = []
    for rho_idx, theta_idx in np.argwhere(accumulator > threshold):
        rho = rho_range[rho_idx]
        theta = theta_range[theta_idx]
        lines.append((rho, theta))

    img_with_lines = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    for rho, theta in lines:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho

        pt1 = (int(x0 + width * (-b)), int(y0 + height * (a)))
        pt2 = (int(x0 - width * (-b)), int(y0 - height * (a)))

        cv2.line(img_with_lines, pt1, pt2, (0, 255, 0), 1)

    display_image(img_with_lines)

root = tk.Tk()
root.title("Custom Hough Transform GUI")
root.geometry("800x600")

btn_upload = tk.Button(root, text="Upload and Detect Edges", command=upload_and_preprocess_image)
btn_upload.pack()

btn_apply_hough = tk.Button(root, text="Apply Custom Hough Transform", command=apply_custom_hough_transform)
btn_apply_hough.pack()

displayed_img = tk.Label(root)
displayed_img.pack()

root.mainloop()
